In [1]:
!pip install python-dotenv --break-system-packages
!pip install boto3 --break-system-packages
!pip install pymediainfo --break-system-packages

In [7]:
import os
from dotenv import load_dotenv
import io
import pathlib
import math
from scipy.io.wavfile import write
import numpy as np
import boto3
from pymediainfo import MediaInfo
import librosa as lb

In [3]:
aws_access_key_id = os.environ.get("aws_access_key_id")
aws_secret_access_key = os.environ.get("aws_secret_access_key")
aws_bucket_name = 'youtubedl'

https://knowledgebase.wasabi.com/hc/en-us/articles/115002579891-How-do-I-use-AWS-SDK-for-Python-boto3-with-Wasabi

The service URL (endpoint_url) depends on the storage region of the bucket:
https://knowledgebase.wasabi.com/hc/en-us/articles/360015106031-What-are-the-service-URLs-for-Wasabi-s-different-regions-

In [4]:
s3 = boto3.client('s3',
                  endpoint_url='https://s3.eu-central-2.wasabisys.com',
                  aws_access_key_id=aws_access_key_id,
                  aws_secret_access_key=aws_secret_access_key)

In [5]:
# return list of audio files for of given folder
def get_audio_files(path: str, exclude_dirs: list = []):
    filtered_files=[]
    for root, dirs, files in os.walk(path, topdown=True):
        if root not in exclude_dirs:
            for file in files:
                file_path = root + "/" + file
                fileInfo = MediaInfo.parse(file_path)
                for track in fileInfo.tracks:
                    if track.track_type == "Audio":
                        filtered_files.append(file_path)
    return filtered_files

In [6]:
dir = '/root/data/announcement_tracker/youtube'
file_paths = get_audio_files(path=dir)

In [7]:
target_sr: int = 16000

def load_audio_16k_mono(file_path):
    y, sr = lb.load(file_path)
    # Convert to mono
    y = lb.to_mono(y=y)
    # set sampling rate
    y = lb.resample(y=y, orig_sr=sr, target_sr=target_sr)
    return y

In [21]:
target_track_length = 2.7*60

def slice_audio(y: np.array):
    duration_total = lb.core.get_duration(y=y, sr=target_sr)
    n_slices=round(duration_total/target_track_length)
    n_slices= 1 if n_slices==0 else n_slices
    slice_size= len(y)//n_slices #1 if n_slices==0 else len(y)//n_slices
    slices = []
    #Slicing
    for i in np.arange(0, n_slices*slice_size, slice_size):
        slices.append(y[i:i+slice_size-1])
    return np.array(slices)

In [23]:
for file_path in file_paths:
    # file_path='/root/data/announcement_tracker/youtube/KvBCzbQLpsk.m4a'
    y = load_audio_16k_mono(file_path)
    print(file_path)
    audio_slices=slice_audio(y)
    for i, audio_slice in enumerate(audio_slices):
        wav_object = io.BytesIO(bytes())
        write(wav_object, target_sr, audio_slice)
        dir_path_rel, file_name=os.path.split(file_path[len(dir)+1:])
        file_name = os.path.splitext(file_name)[0]
        file_path_rel=f'{dir_path_rel}/{file_name}/{file_name}_{i}.wav'
        print(file_path_rel)

/tmp/ipykernel_500908/3554502737.py:4: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = lb.load(file_path)
/usr/local/lib/python3.11/dist-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


/root/data/announcement_tracker/youtube/vTX-Nhs-kUo.m4a
/vTX-Nhs-kUo/vTX-Nhs-kUo_0.wav
/vTX-Nhs-kUo/vTX-Nhs-kUo_1.wav
/vTX-Nhs-kUo/vTX-Nhs-kUo_2.wav
/vTX-Nhs-kUo/vTX-Nhs-kUo_3.wav
/vTX-Nhs-kUo/vTX-Nhs-kUo_4.wav
/vTX-Nhs-kUo/vTX-Nhs-kUo_5.wav
/vTX-Nhs-kUo/vTX-Nhs-kUo_6.wav
/vTX-Nhs-kUo/vTX-Nhs-kUo_7.wav
/vTX-Nhs-kUo/vTX-Nhs-kUo_8.wav
/vTX-Nhs-kUo/vTX-Nhs-kUo_9.wav
/vTX-Nhs-kUo/vTX-Nhs-kUo_10.wav
/vTX-Nhs-kUo/vTX-Nhs-kUo_11.wav
/vTX-Nhs-kUo/vTX-Nhs-kUo_12.wav
/vTX-Nhs-kUo/vTX-Nhs-kUo_13.wav
/vTX-Nhs-kUo/vTX-Nhs-kUo_14.wav
/vTX-Nhs-kUo/vTX-Nhs-kUo_15.wav
/vTX-Nhs-kUo/vTX-Nhs-kUo_16.wav
/vTX-Nhs-kUo/vTX-Nhs-kUo_17.wav
/vTX-Nhs-kUo/vTX-Nhs-kUo_18.wav
/vTX-Nhs-kUo/vTX-Nhs-kUo_19.wav
/vTX-Nhs-kUo/vTX-Nhs-kUo_20.wav
/vTX-Nhs-kUo/vTX-Nhs-kUo_21.wav
/vTX-Nhs-kUo/vTX-Nhs-kUo_22.wav
/vTX-Nhs-kUo/vTX-Nhs-kUo_23.wav
/vTX-Nhs-kUo/vTX-Nhs-kUo_24.wav
/vTX-Nhs-kUo/vTX-Nhs-kUo_25.wav
/vTX-Nhs-kUo/vTX-Nhs-kUo_26.wav
/vTX-Nhs-kUo/vTX-Nhs-kUo_27.wav
/vTX-Nhs-kUo/vTX-Nhs-kUo_28.wav
/vTX-Nhs-kUo/vTX-Nhs-kUo_2

/tmp/ipykernel_500908/3554502737.py:4: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = lb.load(file_path)
/usr/local/lib/python3.11/dist-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


/root/data/announcement_tracker/youtube/cZa7jamQSuM.m4a
/cZa7jamQSuM/cZa7jamQSuM_0.wav
/cZa7jamQSuM/cZa7jamQSuM_1.wav
/cZa7jamQSuM/cZa7jamQSuM_2.wav
/cZa7jamQSuM/cZa7jamQSuM_3.wav
/cZa7jamQSuM/cZa7jamQSuM_4.wav
/cZa7jamQSuM/cZa7jamQSuM_5.wav
/cZa7jamQSuM/cZa7jamQSuM_6.wav
/cZa7jamQSuM/cZa7jamQSuM_7.wav
/cZa7jamQSuM/cZa7jamQSuM_8.wav
/cZa7jamQSuM/cZa7jamQSuM_9.wav
/cZa7jamQSuM/cZa7jamQSuM_10.wav


/tmp/ipykernel_500908/3554502737.py:4: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = lb.load(file_path)
/usr/local/lib/python3.11/dist-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


...

/tmp/ipykernel_500908/3554502737.py:4: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = lb.load(file_path)
/usr/local/lib/python3.11/dist-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


/root/data/announcement_tracker/youtube/EIvr5IGr9Fk.m4a
/EIvr5IGr9Fk/EIvr5IGr9Fk_0.wav
/EIvr5IGr9Fk/EIvr5IGr9Fk_1.wav
/EIvr5IGr9Fk/EIvr5IGr9Fk_2.wav
/EIvr5IGr9Fk/EIvr5IGr9Fk_3.wav
/EIvr5IGr9Fk/EIvr5IGr9Fk_4.wav
/EIvr5IGr9Fk/EIvr5IGr9Fk_5.wav
/EIvr5IGr9Fk/EIvr5IGr9Fk_6.wav
/EIvr5IGr9Fk/EIvr5IGr9Fk_7.wav
/EIvr5IGr9Fk/EIvr5IGr9Fk_8.wav
/EIvr5IGr9Fk/EIvr5IGr9Fk_9.wav
/EIvr5IGr9Fk/EIvr5IGr9Fk_10.wav
/EIvr5IGr9Fk/EIvr5IGr9Fk_11.wav
/EIvr5IGr9Fk/EIvr5IGr9Fk_12.wav
/EIvr5IGr9Fk/EIvr5IGr9Fk_13.wav
/EIvr5IGr9Fk/EIvr5IGr9Fk_14.wav
/EIvr5IGr9Fk/EIvr5IGr9Fk_15.wav
/EIvr5IGr9Fk/EIvr5IGr9Fk_16.wav
/EIvr5IGr9Fk/EIvr5IGr9Fk_17.wav
/EIvr5IGr9Fk/EIvr5IGr9Fk_18.wav
/EIvr5IGr9Fk/EIvr5IGr9Fk_19.wav
/EIvr5IGr9Fk/EIvr5IGr9Fk_20.wav
/EIvr5IGr9Fk/EIvr5IGr9Fk_21.wav
/EIvr5IGr9Fk/EIvr5IGr9Fk_22.wav


/tmp/ipykernel_500908/3554502737.py:4: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = lb.load(file_path)
/usr/local/lib/python3.11/dist-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


: 

: 

In [ ]:
def load_audio_files(file_paths: list):
    for file_path in  file_paths:
        y, sr = lb.load(file_path)
        y = audio_16k_mono(y=y, sr=sr)

In [7]:
#TODO Outsource audio preprocessing to Python module

#TODO convert audio to 16k Hz mono wav file
#TODO slice audio in 3min sections

In [8]:
# file_path = '/root/announcement_tracker/data/faded.wav'
# data = open(file_path, "rb").read()

In [9]:
# write files to S3 storage
# s3.put_object(Body=data, Bucket=aws_bucket_name, Key='faded.wav')